In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy.interpolate import interp1d, interp2d


matplotlib.rc('figure', figsize=(20, 20))

In [ ]:
PATH_TO_WALL_SCAN = '../f1tenth_gym/maps/wall_scan.npz'
RESOLUTION = 0.5

In [ ]:
wall_scan = np.load(PATH_TO_WALL_SCAN)['wall_scan']

In [ ]:
wall_scan.shape

In [ ]:
plt.scatter(wall_scan[:, 0], wall_scan[:, 1], alpha=0.1);

# First, divide the points into two bounds

In [ ]:
ALLOWED_DISTANCE = 2

visited = [0]
not_visited = list(range(1, len(wall_scan)))


while True:
    distances = np.linalg.norm(wall_scan[visited][np.newaxis] - wall_scan[not_visited][:, np.newaxis], axis=2)
    min_distance = np.min(distances)
    if min_distance > ALLOWED_DISTANCE:
        break
        
    closest_idx = np.where(distances == min_distance)[0][0]
    visited.append(not_visited[closest_idx])
    not_visited.remove(not_visited[closest_idx])

In [ ]:
left_bound = wall_scan[visited]
right_bound = wall_scan[not_visited]

plt.scatter(left_bound[:, 0], left_bound[:, 1], alpha=0.2)
plt.scatter(right_bound[:, 0], right_bound[:, 1], alpha=0.2)
plt.scatter(wall_scan[:, 0], wall_scan[:, 1], alpha=0.1);

# Now, we need to sort them

In [ ]:
def sort_bound(bound):
    # Initially, I used 0 as the starting index but that's region in which the density is too high
    current_idx = len(bound) // 2
    not_visited = list(range(len(bound)))
    not_visited.remove(current_idx)
    indices = []

    while len(not_visited) > 0:
        distances = np.linalg.norm(bound[current_idx] - bound[not_visited], axis=1)
        closest_idx = np.argmin(distances)

        indices.append(current_idx)
        current_idx = not_visited[closest_idx]
        not_visited.remove(current_idx)
    
    return bound[indices]

In [ ]:
left_bound = sort_bound(left_bound)
right_bound = sort_bound(right_bound)

In [ ]:
def get_normalized_progress(bound):
    diffs = np.linalg.norm(np.diff(np.r_[bound, bound[[-1]]], axis=0), axis=1)
    total_distance = diffs.sum()
    return diffs.cumsum() / total_distance, total_distance

In [ ]:
left_normalized_progress, left_total_distance = get_normalized_progress(left_bound)
right_normalized_progress, right_total_distance = get_normalized_progress(right_bound)

In [ ]:
common_kwargs = dict(
    assume_sorted=True,
    fill_value='extrapolate',
)

left_x_coord_fn = interp1d(left_normalized_progress, left_bound[:, 0], **common_kwargs)
left_y_coord_fn = interp1d(left_normalized_progress, left_bound[:, 1], **common_kwargs)

right_x_coord_fn = interp1d(right_normalized_progress, right_bound[:, 0], **common_kwargs)
right_y_coord_fn = interp1d(right_normalized_progress, right_bound[:, 1], **common_kwargs)

In [ ]:
num_steps = 1000
time_steps = np.arange(num_steps + 1) / num_steps

In [ ]:
interior = np.c_[left_x_coord_fn(time_steps), left_y_coord_fn(time_steps)]
exterior = np.c_[right_x_coord_fn(time_steps), right_y_coord_fn(time_steps)]
plt.scatter(interior[:, 0], interior[:, 1], alpha=0.2);
plt.scatter(exterior[:, 0], exterior[:, 1], alpha=0.2);

In [ ]:
pd.DataFrame(interior).to_csv('interior.csv', index=False, header=None)
pd.DataFrame(exterior).to_csv('exterior.csv', index=False, header=None)